In [17]:
from importlib import import_module
import pandas as pd 
from datetime import datetime, timedelta
from os import listdir
import plotly.express as px
from os.path import isfile, join
import numpy as np
import sys
sys.path.insert(1, 'C:/Users/maperezr/proyectos/fs-emails')
from adm_email import ADM_MAIL
from input_planillas import f_miguel as fm
dt_string = datetime.today().strftime('%d%m%y')
pd.set_option('display.max_columns', 500)

In [18]:
fecha_inicio = "2022-08-23"
fecha_final = "2022-08-30"
f_inicio = datetime.strptime(fecha_inicio, '%Y-%m-%d')
f_final = datetime.strptime(fecha_final, '%Y-%m-%d')

In [19]:
lista_fechas = [(f_inicio + timedelta(days=d)).strftime('%y%m%d') for d in range((f_final - f_inicio).days + 1)] 

In [20]:
with open('config\path.txt', "r") as tf:
        path = tf.read()
        path = path.replace("\\","/")
tf.close()
alertas = pd.DataFrame()
for i in listdir(path):
        files_store= pd.DataFrame()
        if i.startswith('JPPs - Análisis de notas crédito - '):
                new_path= f"{path}/{i}"
                files_names = [f for f in listdir(new_path) if isfile(join(new_path, f))]
                for j in (files_names):
                        if j.endswith('.xlsx'):
                                for i in lista_fechas:
                                        if j.startswith(i):
                                                if '_cm' in j:
                                                        pass
                                                else:
                                                        file = pd.read_excel(f'{new_path}/{j}',sheet_name="Alertas",dtype=str)
                                                        file["fecha"] = i
                                                        files_store = pd.concat([files_store,file])
        alertas = pd.concat([alertas, files_store])   


In [21]:
alertas_x_hora = alertas.loc[alertas['Tipo alerta'] == 'Alerta x Hora']
alertas_x_monto = alertas.loc[alertas['Tipo alerta'] == 'Alerta x monto']

In [22]:

alertas_x_hora["fecha"] =  pd.to_datetime(alertas_x_hora.fecha, yearfirst=True)
al_x_hora = alertas_x_hora.loc[(alertas_x_hora["fecha"] >= fecha_inicio) & (alertas_x_hora["fecha"] <= fecha_final)].reset_index(drop=True)
al_x_hora["Grabación?"] = al_x_hora["Grabación?"].str.upper()
al_x_hora["Grabación?"] = al_x_hora["Grabación?"].fillna("nan")
al_x_hora.loc[(al_x_hora["Grabación?"].str.contains("NO")) | (al_x_hora["Grabación?"].notna()) & (al_x_hora["Cliente?"].notna()) & (al_x_hora["Producto?"].notna()),"Indicador"] = "Diligenciado"
al_x_hora.loc[al_x_hora["Grabación?"] =="nan", "Grabación?" ] = np.nan
print(f'cantidad de alertas generadas en el corte: {al_x_hora.shape[0]}')
al_x_hora.loc[(al_x_hora["Grabación?"].notna()) & (al_x_hora["Cliente?"].notna()) & (al_x_hora["Producto?"].notna()), "Indicador"] = "Diligenciado"
al_x_hora.loc[al_x_hora["Indicador"] != "Diligenciado", "Indicador"] = "No diligenciado"
fig_al_x_hora = al_x_hora.groupby(["Desc_local","Indicador"]).agg({"Cautoriza":"nunique",}).reset_index().sort_values("Cautoriza", ascending=False)
h_fig = px.bar(fig_al_x_hora, x="Desc_local", y="Cautoriza",color="Indicador",text="Cautoriza",color_discrete_map = {'No diligenciado':'rgb(170, 57, 57)','Diligenciado':'rgb(45, 136, 45)'}, title=f"Cantidad de alertas por hora revisadas por tienda <br>Corte: {fecha_inicio} hasta el {fecha_final}", labels={"Desc_local":"Local","Cautoriza":"Cantidad NCs"}) 
h_fig.update_layout(xaxis_categoryorder = 'total descending',legend=dict(yanchor="top", y=0.95, xanchor="left", x=0.7))
h_fig.write_image(f"output/{dt_string}alerta_horas.jpg", height = 600, width = 1000)
h_fig.show()
al_x_hora.to_excel(f"output/{dt_string}_alertas_x_hora.xlsx", index=False)

cantidad de alertas generadas en el corte: 43


C:\Users\maperezr\AppData\Local\Temp\ipykernel_15648\3762780082.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [23]:
with open('config\path.txt', "r") as tf:
        path = tf.read()
        path = path.replace("\\","/")
tf.close()
alertas_x_monto = pd.DataFrame()
for i in listdir(path):
        files_store= pd.DataFrame()
        if i.startswith('JPPs - Análisis de notas crédito - '):
                new_path= f"{path}/{i}"
                files_names = [f for f in listdir(new_path) if isfile(join(new_path, f))]
                for j in (files_names):
                        if j.endswith('.xlsx'):
                                for i in lista_fechas:
                                        if j.startswith(i):
                                                if '_cm' in j:
                                                        pass
                                                else:
                                                        file = pd.read_excel(f'{new_path}/{j}',sheet_name="Alertas",dtype=str)
                                                        file["fecha"] = i
                                                        files_store = pd.concat([files_store,file])
        alertas_x_monto = pd.concat([alertas_x_monto, files_store]) 

Alertas x monto despues de ultimo informe (20/feb/22)

In [24]:
alertas_x_monto["fecha"] =  pd.to_datetime(alertas_x_monto.fecha, yearfirst=True)
al_x_monto = alertas_x_monto.loc[(alertas_x_monto["fecha"] >= fecha_inicio) & (alertas_x_monto["fecha"] <= fecha_final)].reset_index(drop=True)
al_x_monto["Grabación?"] = al_x_monto["Grabación?"].str.upper()
al_x_monto["Grabación?"] = al_x_monto["Grabación?"].fillna("nan")
al_x_monto.loc[(al_x_monto["Grabación?"].str.contains("NO")) | (al_x_monto["Grabación?"].notna()) & (al_x_monto["Cliente?"].notna()) & (al_x_monto["Producto?"].notna()),"Indicador"] = "Diligenciado"
al_x_monto.loc[al_x_monto["Grabación?"] =="nan", "Grabación?" ] = np.nan
print(f'cantidad de alertas generadas en el corte: {al_x_monto.shape[0]}')
al_x_monto.loc[(al_x_monto["Grabación?"].notna()) & (al_x_monto["Cliente?"].notna()) & (al_x_monto["Producto?"].notna()), "Indicador"] = "Diligenciado"
al_x_monto.loc[al_x_monto["Indicador"] != "Diligenciado", "Indicador"] = "No diligenciado"
fig_al_x_monto = al_x_monto.groupby(["Desc_local","Indicador"]).agg({"Cautoriza":"nunique",}).reset_index().sort_values("Cautoriza", ascending=False)
h_fig = px.bar(fig_al_x_monto, x="Desc_local", y="Cautoriza",color="Indicador",text="Cautoriza",color_discrete_map = {'No diligenciado':'rgb(170, 57, 57)','Diligenciado':'rgb(45, 136, 45)'}, title=f"Cantidad de alertas por monto por tienda <br>Corte:{fecha_inicio} hasta el {fecha_final}", labels={"Desc_local":"Local","Cautoriza":"Cantidad NCs"}) 
h_fig.update_layout(xaxis_categoryorder = 'total descending',legend=dict(yanchor="top", y=0.95, xanchor="left", x=0.7))
h_fig.write_image(f"output/{dt_string}alerta_monto.jpg", height = 600, width = 1000)
h_fig.show()
al_x_monto.to_excel(f"output/{dt_string}alertas_x_monto.xlsx", index=False)

cantidad de alertas generadas en el corte: 3175


Calculos


In [25]:
hora_no_d = al_x_hora.loc[al_x_hora.Indicador == "No diligenciado", "Cautoriza" ].nunique()
hora_d = al_x_hora.loc[al_x_hora.Indicador == "Diligenciado", "Cautoriza" ].nunique()
monto_no_d = al_x_monto.loc[al_x_monto.Indicador == "No diligenciado", "Cautoriza" ].nunique()
monto_d = al_x_monto.loc[al_x_monto.Indicador == "Diligenciado", "Cautoriza" ].nunique()
total_alarmas = al_x_hora.shape[0] + al_x_monto.shape[0]
cantidad_diligenciados = hora_d + monto_d
print(f"Cantidad total de alarmas por hora: {al_x_hora.shape[0]}")
print(f"Cantidad total de alarmas por monto: {al_x_monto.shape[0]}")
print(f"Cantidad total de alarmas: {total_alarmas}")
print(f"Cantidad total no diligenciado: {hora_no_d + monto_no_d}")
print(f"Cantidad diligenciados: {cantidad_diligenciados}")
print(f"Porcentage de alertas diligenciadas: {(cantidad_diligenciados*100)/total_alarmas}")
alertas_hora = al_x_hora.shape[0]
alertas_montos = al_x_monto.shape[0]
alertas_totales = hora_no_d + monto_no_d


Cantidad total de alarmas por hora: 43
Cantidad total de alarmas por monto: 3175
Cantidad total de alarmas: 3218
Cantidad total no diligenciado: 1023
Cantidad diligenciados: 2154
Porcentage de alertas diligenciadas: 66.93598508390305


In [26]:
email_content = f"""<p>Estimados:</p> 
<p>Por favor encuentren a continuación, el análisis de la gestión de diligenciamiento de los reportes de notas créditos según la tienda, para las alarmas por monto y hora, enviadas entre {fecha_inicio} hasta el {fecha_final}.</p>  
<p>En total se generaron {alertas_totales} alertas para las 26 tiendas, las cuales incluye {alertas_hora} alertas por hora y {alertas_montos} alertas por monto. Se evidencia que <strong><u>fueron diligenciadas por las tiendas 1445 alertas que corresponden al 48% del total de alertas</strong></u>.</p> 
<p>Así mismo, en las siguientes gráficas se presentan la cantidad de alertas a revisar por las tiendas y el estado del diligenciamiento correspondiente:</p>  
<img src="cid:alerta_horas"><br></td>
<img src="cid:alerta_montos"><br></td>
<p>El detalle de la información anterior se encuentra adjunto.</p>   
<p>Quedo atento en caso de que surja alguna duda.</p>  
<p>Saludos cordiales,</p> 
<br>
{fm.firma}
"""


In [27]:
img=  {'alerta_horas' : f'output/{dt_string}alerta_horas.jpg', 'alerta_montos':f'output/{dt_string}alerta_monto.jpg'}

In [28]:
print(f'{dt_string}_alertas_x_hora.xlsx')

080922_alertas_x_hora.xlsx


In [29]:
mail = ADM_MAIL()
mail.attach_text(email_content)
mail.attach_image(img)
listado_archive = [f'output\{dt_string}_alertas_x_hora.xlsx', f'output/{dt_string}_alertas_x_monto.xlsx']
mail.attach_archives(listado_archive)

In [30]:
mail.erase_head()

In [31]:
mail.head_mesagge('usuario@falabella.com.co','usuario@falabella.com.co',f'Reporte diligenciamiento alertas NC - {fecha_inicio} hasta el {fecha_final}','usuario@falabella.com.co')

In [32]:
mail.send_mesaje()

TypeError: string payload expected: <class 'list'>

tiendas

In [ ]:
tienda == 'ACQUA', |
tienda == 'TIENDA ALEGRA'    
tienda == 'ARKADIA', |
tienda == 'BUENAVISTA', |
tienda == 'CACIQUE', | 
tienda == 'CARACOLI', | 
tienda == 'CENTRO MAYOR', | 
tienda == 'COLINA', | 
tienda == 'DIVER PLAZA', | 
tienda == 'EL CASTILLO', | 
tienda == 'FONTANAR', |
tienda == 'GALERIAS', |
tienda == 'HAYUELOS', | 
tienda == 'JARDIN PLAZA', | 
tienda == 'LA CAROLA', | 
tienda == 'LA FELICIDAD', | 
tienda == 'PARQUE ARBOLEDA', | 
tienda == 'PLAZA CENTRAL', | 
tienda == 'PRIMAVERA', | 
tienda == 'SAN DIEGO', | 
tienda == 'SANTA FE', | 
tienda == 'SANTAFE MEDELLIN', | 
tienda == 'SUBA', | 
tienda == 'TITAN', | 
tienda == 'UNICENTRO', |
tienda == 'VILLA COUNTRY', | 
tienda == 'WTC CALI'

SyntaxError: invalid syntax (1909356706.py, line 1)

In [ ]:
tienda == 'EL CASTILLO' | tienda == 'FONTANAR' |tienda == 'GALERIAS' |tienda == 'HAYUELOS' | tienda == 'JARDIN PLAZA' | tienda == 'LA CAROLA' | tienda == 'LA FELICIDAD' | tienda == 'PARQUE ARBOLEDA' | tienda == 'PLAZA CENTRAL' | tienda == 'PRIMAVERA' | tienda == 'SAN DIEGO' | tienda == 'SANTA FE' | tienda == 'SANTAFE MEDELLIN' | tienda == 'SUBA' | tienda == 'TITAN' | tienda == 'UNICENTRO' |tienda == 'VILLA COUNTRY' | tienda == 'WTC CALI'